In [1]:
from known_boundary.GP import optimise,optimise_warp
from known_boundary.utlis import Trans_function, get_initial_points
from known_boundary.acquisition_function import EI_acquisition_opt,MES_acquisition_opt,Warped_TEI2_acquisition_opt

/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import GPy
import torch
import botorch
from botorch.test_functions import Ackley,Levy,Beale,Branin,Hartmann,Rosenbrock
from botorch.utils.transforms import unnormalize,normalize

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

In [6]:
fun = Levy(dim=4,negate=False)
dim = fun.dim
bounds = fun.bounds
standard_bounds=np.array([0.,1.]*dim).reshape(-1,2)
 
n_init = 4*dim
iter_num = 10*dim
N=1

fstar = 0.
fun = Trans_function(fun,fstar,min=True)

# GP+EI

In [12]:
BO_EI = []

for exp in range(N):
    
    print(exp)

    seed = exp

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)

    #torch.manual_seed(seed)

    best_record = [Y_BO.min().item()]

    for i in range(iter_num):
        
            train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
            train_X = normalize(X_BO, bounds)
            
            minimal = train_Y.min().item()
            
            train_Y = train_Y.numpy()
            train_X = train_X.numpy()
            
            # train the GP
            res = optimise(train_X,train_Y)
            print('lengthscale is: ',np.sqrt(res[0])) 
            print('variance is: ',res[1])
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
            m = GPy.models.GPRegression(train_X, train_Y,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))

            standard_next_X = EI_acquisition_opt(m,bounds=standard_bounds,f_best=minimal)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())
            #print(best_record)
            
    best_record = np.array(best_record)+fstar 
    BO_EI.append(best_record)

0
lengthscale is:  0.3273816309516063
variance is:  1.060748514931474
lengthscale is:  0.27324993082608906
variance is:  0.9250767392903296
lengthscale is:  0.2946478070488695
variance is:  0.9487288320452513
lengthscale is:  0.3182579523888689
variance is:  1.0021624390296173
lengthscale is:  0.3330901006023945
variance is:  1.0444640789161295
lengthscale is:  0.3406680061897725
variance is:  1.0704578505817761
lengthscale is:  0.350083785753464
variance is:  1.1070946595167939
lengthscale is:  0.36650501401995966
variance is:  1.182604357112496
lengthscale is:  0.37040345036994177
variance is:  1.203645692772834
lengthscale is:  0.357129349326751
variance is:  1.1646754256676286
lengthscale is:  0.3733881368080188
variance is:  1.2452299153761208
lengthscale is:  0.37551465989646005
variance is:  1.2595212429448845
lengthscale is:  0.38748148945635924
variance is:  1.3345447905582104
lengthscale is:  0.39287096452699904
variance is:  1.387014594303233
lengthscale is:  0.3956853400852

# GP+MES

In [7]:
BO_MES = []

for exp in range(N):
    
    print(exp)

    seed = exp
    

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)
    
    fstar_mes = 0.
    
    #torch.manual_seed(seed)

    best_record = [Y_BO.min().item()]

    for i in range(iter_num):

            print(i)
            
            train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
            train_X = normalize(X_BO, bounds)
            
            
            fstar_standard = (fstar_mes - Y_BO.mean()) / Y_BO.std()
            fstar_standard = fstar_standard.item()
            
            train_Y = train_Y.numpy()
            train_X = train_X.numpy()
            
            # train the GP
            res = optimise(train_X,train_Y)
            print('lengthscale is: ',np.sqrt(res[0])) 
            print('variance is: ',res[1])
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
            m = GPy.models.GPRegression(train_X, train_Y,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))

            standard_next_X = MES_acquisition_opt(m,standard_bounds,fstar_standard)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())

            
    best_record = np.array(best_record)+fstar 
    BO_MES.append(best_record)

0
0
lengthscale is:  0.3592089802631885
variance is:  1.083557308779189
1
lengthscale is:  0.3180674399152756
variance is:  0.9707110152337215
2
lengthscale is:  0.3584815692962038
variance is:  1.0675060115009907
3
lengthscale is:  0.22934538413535813
variance is:  0.8876830473134438
4
lengthscale is:  0.250624270033689
variance is:  0.918243504138327
5
lengthscale is:  0.2678940684663105
variance is:  0.9542471124681159
6
lengthscale is:  0.26794543450422453
variance is:  0.9787863784221025
7
lengthscale is:  0.2796949072810095
variance is:  1.0423604164107225
8
lengthscale is:  0.2635979055483248
variance is:  1.0162957713111078
9
lengthscale is:  0.2699507893246039
variance is:  1.0675017717087583
10
lengthscale is:  0.27745995489203346
variance is:  1.1152558138478164
11
lengthscale is:  0.25762476151927133
variance is:  1.1478156265386064
12
lengthscale is:  0.2325097395191491
variance is:  1.1036975475062645
13
lengthscale is:  0.1784887963702053
variance is:  1.0203923209487704

# log GP+TEI

In [8]:
Warped_BO_TEI2 = []

for exp in range(N):
    
    print(exp)

    seed = exp

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)

    #torch.manual_seed(seed)

    best_record = [Y_BO.min().item()]

    for i in range(iter_num):
            
            print(i)
        
            train_Y = Y_BO.numpy()
            train_X = normalize(X_BO, bounds)
            train_X = train_X.numpy()
            
            # train the GP
            res = optimise_warp(train_X, train_Y)
            lengthscale = np.sqrt(res[0])
            variance = res[1]
            c = res[2]
            
            print('lengthscale is: ', lengthscale)
            print('variance is: ',variance)
            print('c is: ',c)
            
            warp_Y = np.log(train_Y+c)
            mean_warp_Y = np.mean(warp_Y) # use to predict mean
            warp_Y_standard = warp_Y-mean_warp_Y
              
            
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= lengthscale,variance=variance)  #np.sqrt(res[0])
            m = GPy.models.GPRegression(train_X, warp_Y_standard,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))
            
            standard_next_X = Warped_TEI2_acquisition_opt(model=m,bounds=standard_bounds,f_best=best_record[-1],c=c,f_mean=mean_warp_Y)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())
            print(best_record[-1])
            
    best_record = np.array(best_record)+fstar         
    Warped_BO_TEI2.append(best_record)

0
0
lengthscale is:  0.2373631226727596
variance is:  0.8038342366028344
c is:  0.3
2.6433916952740235
1
lengthscale is:  0.2540315825865061
variance is:  0.8413414358899696
c is:  0.3
2.6433916952740235
2
lengthscale is:  0.22930244115574075
variance is:  0.8995058545942405
c is:  0.3
1.2335580189451623
3
lengthscale is:  0.22002014705471962
variance is:  1.112090410027289
c is:  0.3
0.7279083444735936
4
lengthscale is:  0.2640182058642345
variance is:  1.444234784010837
c is:  0.3
0.7279083444735936
5
lengthscale is:  0.1265379864671803
variance is:  1.4191217955731135
c is:  0.3
0.7279083444735936
6
lengthscale is:  0.1112682914689718
variance is:  1.4257952558361835
c is:  0.3
0.7279083444735936
7
lengthscale is:  0.09878714728248937
variance is:  1.5389936551153476
c is:  0.3
0.6906921219880804
8
lengthscale is:  0.09326904403265379
variance is:  1.5775483009739464
c is:  0.3
0.3227107108120453
9
lengthscale is:  0.09380988296898862
variance is:  1.755736394321831
c is:  0.3
0.279